### 1. Import Libraries

In [1]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

In [6]:
# Load reference image and convert to grayscale
reference_image = cv2.imread("my2.png")
reference_gray = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)

In [7]:
# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [8]:
# Detect face in reference image
ref_faces = face_cascade.detectMultiScale(reference_gray, scaleFactor=1.1, minNeighbors=5)
if len(ref_faces) == 0:
    raise Exception("No face found in reference image!")

x, y, w, h = ref_faces[0]
ref_face = reference_gray[y:y+h, x:x+w]

In [9]:
# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        detected_face = gray_frame[y:y+h, x:x+w]

        # Resize detected face to match reference size
        detected_face_resized = cv2.resize(detected_face, (ref_face.shape[1], ref_face.shape[0]))

        # Compute similarity using Structural Similarity Index (SSIM)
        similarity = ssim(ref_face, detected_face_resized)

        threshold = 0.5  # 0–1, higher = more similar
        if similarity > threshold:
            label = f"Match! ({similarity:.2f})"
            color = (0, 255, 0)
        else:
            label = f"No Match ({similarity:.2f})"
            color = (0, 0, 255)

        # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Face Verification (OpenCV)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()